## OpenAI API Simple Prompt Engineering Examples Using ChatGPT

Welcome! This notebook is meant to show some simple prompt engineering examples for those just getting started in the field. We will look at four common tasks associated with typical prompt engineering job requirements to get you on the road to becoming an effective developer:

- Content Generation
- Content Summariziation
- Language Translation
- Content Automation

There are a couple of things to consider as we explore these concepts:
First, we are limiting ourselves to exclusively using OpenAI's API even though there might be better alternatives out there to keep things easy to impelement. 
Second, You will need an OpenAI API key to do these lessons. They give you a $5 credit when you sign up but IT CAN COST MONEY if you aren't careful. 
Just be aware of your usage patterns and you will be fine. You can sign up for your API access here: https://platform.openai.com/ 

Let's get started!


### Content Generation
Assume we work for a book company and they are looking at expanding into the children book market. They don't want to invest a lot of money in authors before they have tested the waters a bit. That's where we come in; our first task is to write a prompt to create a kids short story. The customer wants us to generate a story and make sure the AI outputs an HTML table with the names of the characters from the story and their role in the story so they can put it up on their website. 

In [2]:
# import our packages
import os
import openai

# grab our API key from our environment variables
# NOTE: if you don't have an environment variable set, you can just paste your key in directly by modifying the line below and setting your openai_key to your key
# like this: openai_key = "your key goes here"
openai_key = os.getenv('OPENAI_KEY')

openai.api_key = openai_key

# craft our prompt - this is the text that the AI will use to generate the content
# In this example we use the persona feature to give the AI a little bit of context about the type of content we want it to generate
# Specifically we want it to be a children story teller generate content in the style of Hans Christian Andersen
# This sets the tone and style of the content we want to generate
# Next, we give the AI some direction about the type of content we want it to generate to set the "boundaries" of the content - these are typically given to us by the client
#
# Now we deal with the output
# We clearly indicate where the story should go and where the character and role table should go to make it easy to parse out the content
prompt = """
Act as a children story teller in the style of Hans Christian Andersen using the following process:
1. Make the story no more than 1000 words.
2. Create a story that begins with an opening scene that captures the reader's imagination. 
3. Set the scene by providing a detailed description of the setting and atmosphere, including the time of day, the weather, and any other relevant details that help to immerse the reader in the story. 
4. Introduce the minor characters in the story, providing enough detail about each character to make them interesting and memorable. 
5. Introduce the main character of the story, describing their physical appearance in detail and giving some insight into their personality and motivations. 
6. Make sure that the story has a clear plot and a satisfying resolution, and that the themes explored in the story are appropriate for a young audience. 
7. Finally, be sure to use vivid language and imagery to bring the story to life and engage the reader's imagination. 

The story can be about anything you want. Be creative!

When you have given me the story, then produce a HTML table with the names of the characters from the story and their role in the story.

Story: <story goes here>

Characters and Roles: <HTML table goes here>

"""
# Now we use the completion.create() method to generate the content and things really get interesting. See my comments on each of the parameters below
response = openai.Completion.create(
    # This is the model we want to use to generate the content - in this case we are using a ChatGPT 3.5 model because it is cheaper to use than the GPT-4 model
    # Useful for prototyping and testing then using the GPT-4 model for later iterations
    # At the time of this writing, for input the GPT-3.5 model costs $0.0015 per 1000 tokens and the GPT-4 model costs $0.03 per 1000 tokens; for output the 
    # GPT-3.5 model costs $0.002 per 1000 tokens and the GPT-4 model costs $0.06 per 1000 tokens
    # Quite a cost differential between the two models so start cheap to prototype and test and then use the GPT-4 model for later on
    model="text-davinci-003",  

    # This is the prompt we created above
    prompt=prompt,  

    # This is the temperature parameter - it controls the randomness of the output or, put another way, how "creative" the AI is
    # The number can be between 0 and 1 - the closer to 0 the less "creative" the AI is and as the number approaches 1 the more "creative" the AI is
    # Since we are creating children's stories we want the AI to be more creative so we set the temperature to 0.7
    # If we were creating something like a legal document we would want the AI to be less creative so we would set the temperature to 0.2 for example
    temperature=0.7,  

    # This is the max_tokens parameter - it controls the length of the output
    # To quote OpenAI: "You can think of tokens as pieces of words, where 1,000 tokens is about 750 words."
    # our prompt is 1000 words so we set the max_tokens to 2000 to give the AI enough room to generate the content
    # But small enough to keep the cost down
    max_tokens=2000,  

    # This is the top_p parameter - it controls the range of words chosen for the output
    # The values are from 0 to 1 - the closer to 0 the less diverse the output and the closer to 1 the more diverse the output
    # For example, if we set top_p to 0.5 then the AI will only use the top 50% of the most likely words
    # But, if we set top_p to 1.0 then the AI will use all of the words
    # Since we are creating children's stories we want the AI to be more diverse so we set the top_p to 1.0
    # If we were creating something like a legal document we would want the AI to be less diverse so we would set the top_p to 0.5 or less, for example
    top_p=1.0,  

    # This is the frequency_penalty parameter - it controls the likelihood of the AI repeating the same words
    # The values are from 0 to 1 - the closer to 0 the more likely the AI will repeat the same words and the closer to 1 the less likely the AI will repeat the same words
    # Since we are creating children's stories we want the AI to be more diverse with less repetition so we set the frequency_penalty to 0.5
    frequency_penalty=0.5,  

    # This is the presence_penalty parameter - it controls the likelihood of the AI repeating the same words as well
    # The values are from 0 to 1 - the closer to 0 the more likely the AI will repeat the same words and the closer to 1 the less likely the AI will repeat the same words
    # Since we are creating children's stories we want the AI to be more diverse with less repetition so we set the presence_penalty to 0.5
    # The difference between the frequency_penalty and the presence_penalty is that the frequency_penalty is applied on a word-by-word basis and the presence_penalty is applied on a sequence-by-sequence basis
    # So, for example, if we set the presence_penalty to 0.5 and the frequency_penalty to 0.5 then the AI will be less likely to repeat the same words AND less likely to repeat the same sequences of words
    presence_penalty=0.5  
)

# Now we grab the response text from the response object output from the completion.create() method
# Notice the choices[0].text - this is because the completion.create() method can return multiple choices
# Which can be useful for some applications but for our purposes we only want the first choice
# For example, if a human were taking the output and choosing the best response then we would want to return all of the choices
response_text = response.choices[0].text

# Print the response text to the console
print(response_text)

# Save the response to a text file for later use
with open("ContentGeneration.txt", "w") as file:
    file.write(response_text)




Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lina. Lina was an adventurous soul who loved exploring the wild forests around her home. Every day she'd spend hours wandering through the trees and admiring the beauty of nature.

One day, while out exploring, Lina stumbled upon an old hut deep in the forest. The hut looked abandoned but when Lina peered through one of its windows she saw something that made her heart beat faster: inside was the most beautiful creature she had ever seen! It had brilliant emerald eyes and shimmering golden fur. 

Lina knew at once that this must be some kind of magical beast, so she bravely opened the door and stepped inside. As soon as she did, the creature spoke to her in a gentle voice: "Welcome child," it said, "I am a phoenix, sent by your grandparents from their realm beyond the stars." 

The phoenix explained that it was here to help her on a journey to find courage and strength within herself - someth